In [0]:
import requests
from io import BytesIO
from pyspark.sql import SparkSession

# Instancia Spark (já existe dentro do notebook Databricks)
spark = SparkSession.builder.getOrCreate()

# Define os caminhos no volume S3 montado via Unity Catalog
ACTUAL_PATH = "/Volumes/raw/data/actual"

# Lista de arquivos para baixar: nome do arquivo e URL direta de download
files_to_download = [
    {
        "filename": "clientes.csv",
        "url": "https://www.kaggle.com/datasets/teocalvo/teomewhy-loyalty-system/download?file=clientes.csv"
    },
    {
        "filename": "produtos.csv",
        "url": "https://www.kaggle.com/datasets/teocalvo/teomewhy-loyalty-system/download?file=produtos.csv"
    },
    {
        "filename": "transacoes.csv",
        "url": "https://www.kaggle.com/datasets/teocalvo/teomewhy-loyalty-system/download?file=transacoes.csv"
    },
    {
        "filename": "transacao_produto.csv",
        "url": "https://www.kaggle.com/datasets/teocalvo/teomewhy-loyalty-system/download?file=transacao_produto.csv"
    }
]

def download_and_save_csv(file_info):
    print(f"Baixando {file_info['filename']} de {file_info['url']} ...")
    response = requests.get(file_info['url'])
    response.raise_for_status()
    
    # Lê o conteúdo CSV em memória com Spark
    data_bytes = BytesIO(response.content)
    
    # Atenção: ajuste o separador conforme tabela
    sep = ";"  # exemplo para seu caso
    
    df = spark.read.csv(data_bytes, header=True, sep=sep)
    
    # Salva direto no volume S3/UC no formato CSV (pode mudar para parquet ou delta)
    output_path = f"{ACTUAL_PATH}/{file_info['filename']}"
    df.write.mode("overwrite").option("header", "true").csv(output_path)
    
    print(f"Arquivo salvo em {output_path}")

# Executa o download e grava para cada arquivo
for file_info in files_to_download:
    download_and_save_csv(file_info)

print("Todos os arquivos baixados e salvos no volume com sucesso.")


In [0]:
# Se o download for permitido como arquivo local
# Tente este fluxo (se seu cluster ainda deixa, mas isso é raro em ambientes Free Edition):

import requests
import os

file_url = "https://www.kaggle.com/datasets/teocalvo/teomewhy-loyalty-system/download?file=clientes.csv"
local_file = "/tmp/clientes.csv"

# Download via requests
resp = requests.get(file_url)
with open(local_file, "wb") as f:
    f.write(resp.content)

# Copie para seu volume S3 montado
dbutils.fs.cp(f"file:{local_file}", "/Volumes/raw/data/actual/clientes.csv")
